In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setAppName('playground').setMaster('local')
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/19 10:58:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql.types import *

import numpy as np
import pandas as pd
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [5]:
def create_dataframe(spark, rdd, schema):
    """
    Generate a DataFrame from a RDD of Rows and a schema.
    We assume the RDD is correctly formatted, no need to check for anything.
    """

    return spark.createDataFrame(rdd, schema)

In [6]:
rdd = spark.sparkContext.parallelize([('1', 'a'), ('2', 'b'), ('3', 'c'), ('4', 'd'), ('5', 'e'), ('6', 'f')])
schema = StructType([StructField('ID', StringType(), True), StructField('letter', StringType(), True)])

result_df = create_dataframe(spark, rdd, schema)
assert result_df.schema == schema
assert result_df.rdd.collect() == rdd.collect()

In [7]:
genders_rdd = spark.sparkContext.parallelize([('1', 'M'), ('2', 'M'), ('3', 'F'), ('4', 'F'), ('5', 'F'), ('6', 'M')])
grades_rdd = spark.sparkContext.parallelize([('1', 5), ('2', 12), ('3', 7), ('4', 18), ('5', 9), ('6', 5)])

genders_schema = StructType([StructField('ID', StringType(), True), StructField('gender', StringType(), True)])
grades_schema = StructType([StructField('ID', StringType(), True), StructField('grade', StringType(), True)])

genders_df = create_dataframe(spark, genders_rdd, genders_schema)
grades_df = create_dataframe(spark, grades_rdd, grades_schema)

genders_df.createOrReplaceTempView('genders')
grades_df.createOrReplaceTempView('grades')

In [8]:
genders_df[genders_df['ID'] > 2].show()

+---+------+
| ID|gender|
+---+------+
|  3|     F|
|  4|     F|
|  5|     F|
|  6|     M|
+---+------+



In [25]:
def mean_grade_per_gender(spark, genders_df, grades_df):
    """
    Given a RDD of studentID to grades and studentID to gender, compute mean grade for each gender returned as paired RDD.
    Assume all studentIDs are present in both RDDs, making inner join possible, no need to check that.
    Schema of output dataframe should bee gender, mean.
    """
    joined = genders_df.join(grades_df, 'ID')
    print(joined.show())
    mean_grade_per_gender = joined.groupBy('gender').agg({'grade': 'mean'})
    return mean_grade_per_gender

In [26]:
grades_df.show()

+---+-----+
| ID|grade|
+---+-----+
|  1|    5|
|  2|   12|
|  3|    7|
|  4|   18|
|  5|    9|
|  6|    5|
+---+-----+



In [27]:
result_df = mean_grade_per_gender(spark, genders_df, grades_df).toPandas()
result_df.columns == ['gender', 'grade']

assert result_df[result_df['gender'] == 'F'].values[0][1] - 11.3 < 0.1
assert result_df[result_df['gender'] == 'M'].values[0][1] - 7.3 < 0.1

+---+------+-----+
| ID|gender|grade|
+---+------+-----+
|  1|     M|    5|
|  2|     M|   12|
|  3|     F|    7|
|  4|     F|   18|
|  5|     F|    9|
|  6|     M|    5|
+---+------+-----+

None


In [29]:
def read_csv(spark, path):
    """
    Create a DataFrame by loading an external csv file. We don't expect any formatting nor processing here. 
    We assume the file has a header, uses " as double quote and , as delimiter. Infer its schema automatically.
    You don't need to raise an exception if the file does not exist.    
    """
    # YOUR CODE HERE
    return spark.read.csv(path, header=True, quote='"', inferSchema=True)

In [31]:
file_path = '../data/FL_insurance_sample.csv'
result_df = read_csv(spark, file_path)

assert result_df.schema == StructType([
    StructField('policyID',IntegerType(),True),
    StructField('statecode',StringType(),True),
    StructField('county',StringType(),True),
    StructField('eq_site_limit',DoubleType(),True),
    StructField('hu_site_limit',DoubleType(),True),
    StructField('fl_site_limit',DoubleType(),True),
    StructField('fr_site_limit',DoubleType(),True),
    StructField('tiv_2011',DoubleType(),True),
    StructField('tiv_2012',DoubleType(),True),
    StructField('eq_site_deductible',DoubleType(),True),
    StructField('hu_site_deductible',DoubleType(),True),
    StructField('fl_site_deductible',DoubleType(),True),
    StructField('fr_site_deductible',IntegerType(),True),
    StructField('point_latitude',DoubleType(),True),
    StructField('point_longitude',DoubleType(),True),
    StructField('line',StringType(),True),
    StructField('construction',StringType(),True),
    StructField('point_granularity',IntegerType(),True)
])

In [32]:
insurance_df = read_csv(spark, '../data/FL_insurance_sample.csv')
insurance_df.createOrReplaceTempView('insurance')

In [33]:
def count_county(spark, insurance_df):
    """
    Return a Pandas a dataframe which contains, for each county, the number of its occurences in the dataset. 
    Schema of the Dataframe should be ['county', 'count']    
    """
    return insurance_df.groupBy('county').count().toPandas()

In [34]:
df = count_county(spark, insurance_df)
result = df.set_index('county').to_dict()['count']

assert result.get('CLAY COUNTY') == 346

In [36]:
# Plot it for fun with bokeh.
data = count_county(spark, insurance_df)

source = ColumnDataSource(data)

hover = HoverTool(tooltips=[
    ("type", "@county"),
    ("count", "@count"),
])

p = figure(x_range=data['county'].values, height=250, title="County counts", tools=[hover])

p.vbar(x='county', top='count', width=0.9, source=source)

p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

In [37]:
spark.stop()